## Get the old and new rods 

In [16]:
class Bar():
    
    def __init__(self, label, length):
        
        self.label    = label
        self.length   = length
        self.has_pair = False

In [17]:
old_bars = []
with open ('old_rod.txt') as file:
    lines = file.read().split('\n')
    
    for i in range(0, len(lines), 4):
        length_list = [float(v) for v in lines[i+2][8:].split(', ')]
        label_list  = lines[i+3][7:].split(', ')

        for label, length in zip(label_list, length_list):
            bar = Bar(label, length)
            old_bars.append(bar)

In [18]:
new_bars = []
with open ('rod.dat') as file:
    lines = file.read().split('\n')
    
    for i in range(0, len(lines), 4):
        length_list = [float(v) for v in lines[i+2][8:].split(', ')]
        label_list  = lines[i+3][7:].split(', ')

        for label, length in zip(label_list, length_list):
            bar = Bar(label, length)
            new_bars.append(bar)

In [19]:
len(new_bars)

178

## Creare fully reusable rod pairs

In [20]:
class Pair():
    
    def __init__(self, old_bar, new_bar):
        
        self.old_bar = old_bar
        self.new_bar = new_bar
        
    def __repr__(self):
        
        return 'old_index: %s, new_index: %s, length: %.2f' % (self.old_bar.label, self.new_bar.label, self.new_bar.length)

In [21]:
print(len(new_bars), len(old_bars))

178 186


In [22]:
tolerance = 1

pair_list = []

for new_bar in new_bars:

    for old_bar in old_bars:
        
        if not old_bar.has_pair and abs(old_bar.length - new_bar.length) <= tolerance:
            
            pair = Pair(old_bar, new_bar)
            old_bar.has_pair = True
            new_bar.has_pair = True
            pair_list.append(pair)
            break

In [23]:
print(len(pair_list))

83


In [24]:
print(len(old_bars))
old_bars =  [b for b in old_bars if not b.has_pair]
print(len(old_bars))

186
103


In [25]:
print(len(new_bars))
new_bars =  [b for b in new_bars if not b.has_pair]
print(len(new_bars))

178
95


## Bin-packing

In [26]:
class Rod(object):
    
    def __init__(self, length, label):
        
        self.length = length
        self.label  = label

class Bin(object):

    def __init__(self, length, old_index):
    
        self.items = []
        self.sum   = 0
        self.length = length
        self.old_index = old_index

    def append(self, item):
        
        self.items.append(item)
        self.sum += item.length

    def result(self, index):
        
        total = self.total_length()
        
        lengths = []
        labels  = []
        place_holder = 'INDEX: %d, ROD_ID: %s, ROD_LENGTH: %.2f \nTOTAL: %.2f'
        
        ph_length = '\nLENGTH: '
        ph_label  = '\nLABEL: '
        
        for item in self.items:
            ph_length += '%.2f, '
            ph_label  += '%s, '
            lengths.append(item.length)
            labels.append(item.label)
            
        if index > 0:
            place_holder = '\n' + place_holder
            
        ph_length = ph_length[:-2]
        ph_label  = ph_label[:-2]
        
        place_holder = place_holder + ph_length + ph_label + '\n'
        return place_holder % tuple([index, self.old_index, self.length, total] + lengths + labels)

    def total_length(self):
        
        total = 0
        
        for item in self.items:
            total += item.length
            
        return total

In [27]:
import operator

new_bars.sort(key=operator.attrgetter('length'), reverse=True)
old_bars.sort(key=operator.attrgetter('length'), reverse=True)

In [28]:
bins = []

old_bar_index = 0

for new_bar in new_bars:
    
    item = Rod(new_bar.length, new_bar.label)

    for bin in bins:

        if bin.sum + item.length <= bin.length:
            bin.append(item)
            break

    else:
        
        old_bar = old_bars[old_bar_index]
        bin = Bin(old_bar.length, old_bar.label)
        bin.append(item)
        bins.append(bin)
        old_bar_index += 1

result = ''
sum_length = 0
for i, bin in enumerate(bins):
    
    result += bin.result(i)
    total = bin.total_length()
    sum_length += total
    
print(result)

INDEX: 0, ROD_ID: 0863, ROD_LENGTH: 1189.13 
TOTAL: 1185.20
LENGTH: 535.75, 534.93, 114.52
LABEL: 0036, 0236, 4354

INDEX: 1, ROD_ID: 1863, ROD_LENGTH: 970.29 
TOTAL: 933.06
LENGTH: 505.66, 427.40
LABEL: 3863, 2346

INDEX: 2, ROD_ID: 1363, ROD_LENGTH: 831.62 
TOTAL: 831.50
LENGTH: 486.27, 345.23
LABEL: 2036, 0011

INDEX: 3, ROD_ID: 4063, ROD_LENGTH: 830.43 
TOTAL: 828.63
LENGTH: 475.50, 353.13
LABEL: 6263, 2853

INDEX: 4, ROD_ID: 5863, ROD_LENGTH: 768.14 
TOTAL: 765.41
LENGTH: 427.08, 338.33
LABEL: 3136, 1723

INDEX: 5, ROD_ID: 5663, ROD_LENGTH: 715.93 
TOTAL: 712.05
LENGTH: 411.74, 300.31
LABEL: 1122, 2135

INDEX: 6, ROD_ID: 4863, ROD_LENGTH: 615.58 
TOTAL: 612.69
LENGTH: 404.44, 208.25
LABEL: 3862, 4254

INDEX: 7, ROD_ID: 3563, ROD_LENGTH: 596.53 
TOTAL: 591.77
LENGTH: 376.78, 214.99
LABEL: 1035, 1529

INDEX: 8, ROD_ID: 0036, ROD_LENGTH: 539.75 
TOTAL: 538.70
LENGTH: 368.82, 169.88
LABEL: 0611, 2031

INDEX: 9, ROD_ID: 0236, ROD_LENGTH: 538.93 
TOTAL: 536.11
LENGTH: 368.40, 167.71
LAB

## Write

In [29]:
with open('./final_rod.dat', 'w') as file:
    file.write(result)

In [31]:
pair_result = ''

for p in pair_list:
    pair_result += p.__repr__() + '\n'

with open('./final_pair_rod.dat', 'w') as file:
    file.write(pair_result)